In [1]:
using DataFrames, Optim, Revise, Serialization

In [2]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__);

In [3]:
using BLPmodule; const m = BLPmodule;

In [4]:
df = deserialize("jls/df.jls");
pars = deserialize("jls/pars.jls");
ec = deserialize("jls/ec.jls");

---

In [5]:
function g!(storage::Array, θ2::Array)
    storage[:] = m.gmm_grad(θ2, ec, pars, X1)
end

g! (generic function with 1 method)

In [6]:
# linear characteristics (staffing variables):
X1 = vcat(transpose.([ec.firms[i].X for i in eachindex(ec.firms)])...)
Z = X1;

In [7]:
initial_δ = zeros(length(ec.firms), 1);

In [22]:
pars = deserialize("jls/pars.jls");
m.gmm_lm([10.,10.], ec, pars, X1)

obj = 150.80675378431536
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:22)
pars.σ = [10.0, 10.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:23)


150.80675378431536

In [12]:
m.gmm_lm([1.,1.], ec, pars, X1)

obj = 151.19673434313256
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:22)


151.19673434313256

In [23]:
pars.σ

2-element Vector{Float64}:
 1.0
 1.0

In [20]:
m.gmm_grad([2.,2.], ec, pars, X1)

pars.σ[1] = 1.0
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:30)


grad = [-121.22994827721004; -4.122342967979769]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:83)


2×1 Matrix{Float64}:
 -121.22994827721004
   -4.122342967979769

In [16]:
pars = deserialize("jls/pars.jls");
m.gmm_grad_alt([1.,1.], ec, pars, X1)

gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsuppor

110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facs

110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facs

110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facsupport/zhli/blp-quantities/BLPmodule/gmm.jl:110)
gradj[jj] = [0.0, 0.0]
(in /users/facs

2-element Vector{Float64}:
 0.0
 0.0

In [17]:
initial_θ2 = [10.,20.]
pars = deserialize("jls/pars.jls");
res = optimize(θ2 -> m.gmm_lm(θ2, ec, pars, X1), g!, initial_θ2, LBFGS())

δ[1] = 0.0
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:79)


grad = [0.0; 0.0]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:84)
obj = 150.89460765482497
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:22)


 * Status: success

 * Candidate solution
    Final objective value:     1.508946e+02

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = 0.00e+00 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    0
    f(x) calls:    1
    ∇f(x) calls:   1


In [15]:
initial_θ2 = [1.,1.]

pars = deserialize("jls/pars.jls");
res = optimize(θ2 -> m.gmm_lm(θ2, ec, pars, X1), g!, initial_θ2, LBFGS())

θ2 = [1.0, 1.0]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:11)
θ1 = [-0.6906467099492097; -0.23862404002686533]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:19)
obj = 303.2455807486537
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:22)


θ2 = [-7.985913587649435, -2.118980246319886]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:11)
θ1 = [-0.8291268772894276; -0.24072463157591018]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:19)
obj = 399.67644347085337
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:22)


θ2 = [-3.4929567938247175, -0.559490123159943]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:11)
θ1 = [-0.8217679853456918; -0.2264794835658468]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:19)
obj = 391.9805397420546
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:22)


θ2 = [-1.2464783969123587, 0.22025493842002852]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:11)
θ1 = [-0.8188995655069673; -0.2229277626396532]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:19)
obj = 391.15435041142774
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:22)


θ2 = [-0.12323919845617937, 0.6101274692100143]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:11)
θ1 = [-0.8186407541413621; -0.2211988801349134]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:19)
obj = 392.5059495623277
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:22)


...

θ2 = [0.9999999999999818, 0.9999999999999937]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:11)
θ1 = [-0.819518881232087; -0.21397816296919503]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:19)
obj = 402.48604975468714
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:22)


θ2 = [0.9999999999999978, 0.9999999999999992]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:11)
θ1 = [-0.8195262051911228; -0.21383125370650624]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:19)
obj = 402.6942473938014
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:22)


θ2 = [0.9999999999999998, 0.9999999999999999]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:11)
θ1 = [-0.819532895267828; -0.21369713798716572]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:19)
obj = 402.884744680087
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:22)


θ2 = [1.0, 1.0]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:11)
θ1 = [-0.8195390142695097; -0.21357452414933598]
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:19)
obj = 403.0592720680424
(in /mnt/staff/zhli/blp-quantities/BLPmodule/gmm.jl:22)


 * Status: failure (line search failed)

 * Candidate solution
    Final objective value:     4.030593e+02

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 0.0e+00
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = 9.98e+01 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.48e-01 ≰ 0.0e+00
    |g(x)|                 = 7.74e+01 ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1
    f(x) calls:    24
    ∇f(x) calls:   24


In [18]:
size(inv(X1' * X1))

(2, 2)

In [15]:
println(Optim.minimizer(res))
println(Optim.initial_state(res))

[2.0, 3.0]
[2.0, 3.0]


---